# Asynchronous programming with Python
## Module 4 - Compare asynchronous code approaches - multiprocessing

### Agenda:

* Multiprocessing - calculate π value with Monte-Carlo method.
    * Using ProcessPoolExecutor.
    * Trio.
    * AsyncIO.

### Calculating the π value with Monte-Carlo method

Monte Carlo methods, or Monte Carlo experiments, are a broad class of
computational algorithms that rely on repeated random sampling
to obtain numerical results.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Monte_Carlo_method">Wikipedia / Monte Carlo method </a>
</div>

#### The method definition
<div align="center"><img src="../images/Pi_30K.gif" alt="three seagulls" width="200"/></div>

We have a square with a side of length 1, and a quadrant (circular sector) inscribed in it.

The area of the square is 1.

A circul area is calculated as $πr^{2}$, with $r=1$ it gives just $π$. So, the area of the quadrant is $\frac{π}{4}$.

If we drop random points on a square, some of them will land in the quadrant, and some of them will be outside of it.

The ratio of the points inside the quadrant and the total number of points is an estimate of the ratio of the two areas.  That is:

$$\frac{N_{quadrant}}{N_{total}} = \frac{π}{4}$$

$$π = 4 \frac{N_{quadrant}}{N_{total}}$$

All points inside the quadrant meet following condition:

$$x^{2} + y^{2} ≤ 1$$

#### Implementation

Here is a simple (and inefficient) implementation:

In [ ]:
import math
import random

POINTS = 100_000_000


def get_points(n):
    """Get points inside the quadrant."""
    points = 0
    random_ = random.random

    for _ in range(int(n)):
        x = random_()
        y = random_()
        points += (x*x + y*y) <= 1

    return points


def print_result(π):
    """Print out the result of the calculated π value."""
    print(π)
    print("The difference is:", π - math.pi)

In [ ]:
%%time
# This takes half a minute on my machine

print("The π is roughly equal to...")
π = 4 * get_points(POINTS) / POINTS
print_result(π)

👉 *What is the maximum number of random points we can use for such kind of tasks?
Python uses [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister),
one of the most efficient pseudo-random generators,
that has a period of $4.3✕10^{106001}$.
That is, we are good to use Python's `random` module without a risk to
run out of random points.* 👈

#### Parallelization
We can have distribute the points calculation over subprocesses,
then aggregate the numbers.

We will distribute the task across 10 workers, each will provide 1/10 of points.

In [ ]:
SUBTASKS_NUM = 10
SUBTASK_POINTS = POINTS // SUBTASKS_NUM

For the simplest cases you can just go with
[`ProcessPoolExecutor`](https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor),
which handles most of the subprocess-related logic for us.

It has the same interface as `ThreadPoolExecutor` discussed on a
previous module.

In [ ]:
%%time
# On my 8-core machine it takes 10 seconds.

from concurrent.futures import ProcessPoolExecutor

print("The π is roughly equal to...")

with ProcessPoolExecutor() as executor:
    points = sum(
        executor.map(get_points, [SUBTASK_POINTS for _ in range(SUBTASKS_NUM)])
    )


π = 4 * points / POINTS
print_result(π)

### Subprocesses with `async/await` code
The example above showed how code can be executed *asynchronously*
without any async/await syntax.  Though, it's often desired for your
application to keep working with the main thread while waiting for
a response from a subprocess.

`async/await` code can help with this, also for current task is not
necessary.

Both Trio and AsyncIO use interface similar to
[subprocess.Popen](https://docs.python.org/3/library/subprocess.html#popen-constructor).

In order to pass `get_points` function to a subprocess we'll need
some common arguments.

In [ ]:
import inspect
import sys


SUBTASK_SCRIPT_TEMPLATE = """
import random

{func_src}

print(get_points({points_num}))
"""


def get_subprocess_arguments():
    """Provide arguments to run `get_points` from a subprocess.

    The arguments are equivalent to calling a subprocess with
    a shell command

        python -c 'import random
        def get_points(n):
            ...
        print(get_points(1000...))
        '
    """
    func_src = inspect.getsource(get_points)
    script = SUBTASK_SCRIPT_TEMPLATE.format(func_src=func_src, points_num=SUBTASK_POINTS)
    return [sys.executable, "-c", script]

#### Using AsyncIO
AsyncIO has
[two similar functions](https://docs.python.org/3.9/library/asyncio-subprocess.html)
to create a subprocess with `async` interface:
`asyncio.create_subprocess_exec()` and `asyncio.create_subprocess_shell()`.

👉 *Make it a routine to create subprocesses in `exec` mode, not `shell`.
Escaping shell command may be tricky, and it will be even trickier if
your code may be run on Windows.* 👈

👉 *AsyncIO subprocesses have limited support for Windows,
see [here](https://docs.python.org/3.9/library/asyncio-platforms.html#subprocess-support-on-windows)
for details.* 👈

In [ ]:
import asyncio
import os


async def main():
    semaphore = asyncio.Semaphore(os.cpu_count() or 2)

    tasks = [
        asyncio.create_task(run_subtask(semaphore)) for _ in range(SUBTASKS_NUM)
    ]
    points = await asyncio.gather(*tasks)
    return sum(points)


async def run_subtask(semaphore):
    async with semaphore:
        proc = await asyncio.create_subprocess_exec(
            *get_subprocess_arguments(),
            stdout=asyncio.subprocess.PIPE)

        stdout = await proc.stdout.readline()

        await proc.wait()
        return int(stdout.decode().strip())


points = await main()
π = 4 * points / POINTS
print_result(π)

👉 *For current example the subprocess is short-lived, so that even
if the upstream code crashes, we do not end up with orphan subprocesses.
For long-lived subprocesses you may need to add extra cleanup, also
refer to module 3 to see the how clean up after `asyncio.gather()`.* 👈


#### Using Trio
Trio has
[two options](https://trio.readthedocs.io/en/stable/reference-io.html#options-for-starting-subprocesses)
to start a process, the simpler one is
[trio.run_process()](https://trio.readthedocs.io/en/stable/reference-io.html#trio.run_process).

In [ ]:
!pip install trio

In [ ]:
%%time
import os

import trio


async def main():
    points_list = []
    limiter = trio.CapacityLimiter(os.cpu_count() or 2)

    async with trio.open_nursery() as nursery:
        for _ in range(SUBTASKS_NUM):
            nursery.start_soon(run_subtask, points_list, limiter)

    return sum(points_list)


async def run_subtask(points_list, limiter):
    """Calculate the points in a subprocess."""
    async with limiter:
        completed_process = await trio.run_process(
            get_subprocess_arguments(),
            capture_stdout=True
        )
        points_list.append(int(completed_process.stdout.strip()))


print("The π is roughly equal to...")
points = trio.run(main)
π = 4 * points / POINTS
print_result(π)

#### Using Trio - long running process
Spawning a new process is expensive.  Let's spawn a limited number
of processes with
[trio.open_process()](https://trio.readthedocs.io/en/stable/reference-io.html#trio.open_process)
and feed it with the numbers.

In [ ]:
import inspect
import sys


CONTINIOUS_SUBTASK_SCRIPT_TEMPLATE = """
import random

{get_points}

{produce_points}

produce_points()
"""

def produce_points():
    """Read number of points from stdin and print them out."""
    import sys
    print("starting a subprocess", file=sys.stderr)

    for line in sys.stdin:
        print("got a line:", line, file=sys.stderr)

        num = int(line.strip())
        result = get_points(num)

        print("printing out a result:", result, file=sys.stderr)
        print(result)

    print("exiting", file=sys.stderr)


def get_continuous_subprocess_arguments():
    """Provide arguments to continuously run `get_points`.

    The arguments are equivalent to calling a subprocess with
    a shell command

        python -c 'import random
        def get_points(n):
            ...
        produce_points()
        '
    """
    get_points_src = inspect.getsource(get_points)
    produce_points_src = inspect.getsource(produce_points)
    script = CONTINIOUS_SUBTASK_SCRIPT_TEMPLATE.format(
        get_points=get_points_src, produce_points=produce_points_src,
    )
    return [sys.executable, "-u", "-c", script]


In [ ]:
%%time

import os
import subprocess

import trio


async def main():
    points_list = []
    num_of_workers = min(SUBTASKS_NUM, os.cpu_count() or 2)
    send_channel, receive_channel = trio.open_memory_channel(num_of_workers)

    async with trio.open_nursery() as nursery:
        async with receive_channel:
            for _ in range(num_of_workers):
                nursery.start_soon(
                    run_subtask, points_list, receive_channel.clone(), nursery
                )

        async with send_channel:
            for _ in range(SUBTASKS_NUM):
                await send_channel.send(SUBTASK_POINTS)

    return sum(points_list)


async def run_subtask(points_list, receive_channel, nursery):
    """Calculate the points in a subprocess."""
    process = await trio.open_process(
        get_continuous_subprocess_arguments(),
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
    )

    async with receive_channel:
        nursery.start_soon(populate_points, process, points_list)
        async for points_num in receive_channel:
            await process.stdin.send_all(f"{points_num}\n".encode())
        await process.stdin.aclose()


async def populate_points(process, points_list):
    """Read stdout and populate the points list with numbers."""
    async with process:
        buffer = b""
        async for data in process.stdout:
            buffer += data
        points_list.extend(int(s.strip()) for s in buffer.splitlines())


print("The π is roughly equal to...")
points = trio.run(main)
π = 4 * points / POINTS
print_result(π)

👉 *Note of following: (1) `-u` option is use; (2) writting in `stdin` and reading from `stdout` is done concurrently to prevent deadlocks.* 👈

<span style="font-size: x-large">Add your code below:</span>